In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url = f"https://jawatengah.online/category/politik/page/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('header',{"class": "post-title entry-header"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        link="https://jawatengah.online"+link
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 21 links from page 1
Total Links: 21


In [5]:

def scrape_url(url):
    try:
        response = requests.get(url,headers={'User-Agent': 'Chrome/117.0.0.0'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('strong', {"itemprop": "author"})
            if author_elem:
                author_text = author_elem.find('a')
                author_text = author_text.text
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "time"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content entry-content cf"})
            
            if body_elem:
                content_elem = body_elem.find_all('div')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jateng',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Failed to retrieve data from https://jawatengah.online/02/11/2022/anggota-dprd-jateng-sentil-ganjar-sering-absen-saat-paripurna/
Failed to retrieve data from https://jawatengah.online/18/08/2023/dpd-golkar-jateng-ikuti-komando-ketua-umum-airlangga-dukung-prabowo-subianto-di-pilpres-2024%ef%bf%bc/
Failed to retrieve data from https://jawatengah.online/06/09/2023/rakornas-japelidi-kuatkan-komitmen-literasi-digital/
Failed to retrieve data from https://jawatengah.online/18/08/2023/dpd-golkar-jateng-ikuti-komando-ketua-umum-airlangga-dukung-prabowo-subianto-di-pilpres-2024%ef%bf%bc/
Failed to retrieve data from https://jawatengah.online/04/09/2023/meriah-jalan-santai-di-sukorame-bs-ikut-serahkan-hadiah-sepeda/
Failed to retrieve data from https://jawatengah.online/18/08/2023/pakar-minta-bawaslu-jateng-usut-kehadiran-pj-bupati-walikota-di-konsolidasi-pdip-semarang%ef%bf%bc/
Failed to retrieve data from https://jawatengah.online/13/10/2022/kawal-pemilu-2024-ki-pusat-sampaikan-ini/
Failed to 

In [7]:
df = pd.DataFrame(results)
print(len(results))
df.head(5)

9


,title,author,category,date,content,region,link
0,Ketulusan Sosial Didi Kempot,Bayu Wardhana,Category not found,02/08/2020,Content not found,jateng,https://jawatengah.online/02/08/2020/ketulusan...
1,Kota Kudus Tempo Doeloe,JT Online,Category not found,21/01/2020,Content not found,jateng,https://jawatengah.online/21/01/2020/kota-kudu...
2,Pakar Minta Bawaslu Jateng Usut Kehadiran Pj B...,JT Online,Category not found,1 month ago,Content not found,jateng,https://jawatengah.online/18/08/2023/pakar-min...
3,"Tak Hanya CCTV dan Mobile ETLE, Ditlantas Pold...",JT Online,Category not found,11/10/2022,Content not found,jateng,https://jawatengah.online/11/10/2022/tak-hanya...
4,"Pendaftaran Panwascam Dibuka, Ini Persyaratanya",JT Online,Category not found,16/09/2022,Content not found,jateng,https://jawatengah.online/16/09/2022/pendaftar...


In [8]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/jateng_online_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')